In [1]:
import pandas as pd
import re
import pymorphy2

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from sklearn import tree

In [3]:
df = pd.read_csv('lenta-ru-train.csv')

In [4]:
df.head()

,title,text,topic,topic_label
0,"Инфляция в январе 2006 года составит 2,6 процента","Глава Росстата Владимир Соколин заявил, что в ...",Экономика,0
1,Никита Михалков учредил День российского кино,У российских кинематографистов появится новый...,Культура,3
2,Марко Матерацци вернется в строй к матчу с ЦСКА,"Медицинский штаб миланского ""Интера"" обнародов...",Спорт,1
3,"Определены лауреаты премии ""Книга года""","Премии ""Книга года"" в 13 номинациях вручены на...",Культура,3
4,Гол Роналду со штрафного спас португальцев от ...,Сборная Португалии сыграла вничью с командой И...,Спорт,1


In [5]:
df.topic.value_counts(normalize=True)

Экономика          0.307646
Спорт              0.250096
Культура           0.208443
Наука и техника    0.205113
Бизнес             0.028703
Name: topic, dtype: float64

In [6]:
df.isna().sum()

title          0
text           1
topic          0
topic_label    0
dtype: int64

In [7]:
df[df.text.isna()]['title'].values

array(['Гребенщиков заработал 10\xa0тысяч рублей в\xa0омском подземном переходе'],
      dtype=object)

In [8]:
df = df[~df.text.isna()]

In [9]:
df.shape

(206636, 4)

In [25]:
rs = 1915

### Базовая обработка

In [10]:
text_transformer = TfidfVectorizer()

In [11]:
text = text_transformer.fit_transform(df['text'])

In [12]:
text

<206636x567399 sparse matrix of type '<class 'numpy.float64'>'
	with 27370083 stored elements in Compressed Sparse Row format>

In [13]:
X_train, X_test, y_train, y_test = train_test_split(text, df['topic_label'], test_size=0.20, random_state=rs)

In [14]:
X_train.shape

(165308, 567399)

In [15]:
X_test.shape

(41328, 567399)

In [19]:
clf = tree.DecisionTreeClassifier()

In [20]:
clf.fit(X_train,y_train)

DecisionTreeClassifier()

In [21]:
pred = clf.predict(X_test)
print('Качество модели по метрике F1', f1_score(y_test,pred,average='weighted'))

Качество модели по метрике F1 0.8633452789745651


### Изменим токенизатор

In [22]:
stop_russian = stopwords.words('russian')

In [23]:
stop_russian

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

In [26]:
text_transformer = TfidfVectorizer(stop_words=stop_russian, ngram_range=(1,1), lowercase=True, max_features=10000 )
text = text_transformer.fit_transform(df['text'])
X_train, X_test, y_train, y_test = train_test_split(text, df['topic_label'], test_size=0.2, random_state=rs)

In [27]:
X_train.shape

(165308, 10000)

In [28]:
clf2 = tree.DecisionTreeClassifier()
clf2.fit(X_train,y_train)

DecisionTreeClassifier()

In [29]:
pred2 = clf2.predict(X_test)
print('Качество модели по метрике F1 (токенизатор настроен)', f1_score(y_test,pred2,average='weighted'))

Качество модели по метрике F1 (токенизатор настроен) 0.8787745921040077


### приведем слова к первой форме

In [30]:
pattern = r'[^А-Яа-я]+'

In [39]:
list = [re.sub(pattern, ' ', i) for i in ['проверка weyweywt iweiwue  238238 ? 123 тест']]

In [40]:
list

['проверка тест']

In [41]:
# функция удаления пунктуации и цифр
def remove_trash(list): 
    pattern = r'[^А-Яа-я]+'
    try:
      list = [re.sub(pattern, ' ', i) for i in list] 
      print (list)
    except Exception as e:
      print(e)
    return list

In [42]:
df['text_clean'] = remove_trash(df['text'])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [44]:
df[['text','text_clean']].head()

,text,text_clean
0,"Глава Росстата Владимир Соколин заявил, что в ...",Глава Росстата Владимир Соколин заявил что в я...
1,У российских кинематографистов появится новый...,У российских кинематографистов появится новый ...
2,"Медицинский штаб миланского ""Интера"" обнародов...",Медицинский штаб миланского Интера обнародовал...
3,"Премии ""Книга года"" в 13 номинациях вручены на...",Премии Книга года в номинациях вручены на Моск...
4,Сборная Португалии сыграла вничью с командой И...,Сборная Португалии сыграла вничью с командой И...


In [45]:
df = df[~df.text_clean.isna()].copy()

In [ ]:
#токенизатор удалил мусор

In [46]:
text_transformer = TfidfVectorizer(stop_words=stop_russian, ngram_range=(1,1), lowercase=True, max_features=10000 )
text = text_transformer.fit_transform(df['text_clean'])
X_train, X_test, y_train, y_test = train_test_split(text, df['topic_label'], test_size=0.2, random_state=rs)

In [47]:
clf3 = tree.DecisionTreeClassifier()
clf3.fit(X_train,y_train)
pred3 = clf3.predict(X_test)
print('Качество модели по метрике F1 (токенизатор настроен и убран мусор)', f1_score(y_test,pred3,average='weighted'))

Качество модели по метрике F1 (токенизатор настроен и убран мусор) 0.8750528908287197


### попробуем лемматизацию

In [48]:
morph = pymorphy2.MorphAnalyzer()

In [49]:
# функция лемматизации
def lemmatize(row):
    t = []
    text = row['text_clean']
    for word in text.split():
        if len(word)<=3:
            continue
        p = morph.parse(word)[0]
        t.append(p.normal_form)
    return " ".join(t)

In [50]:
df.text

0         Глава Росстата Владимир Соколин заявил, что в ...
1         У российских кинематографистов  появится новый...
2         Медицинский штаб миланского "Интера" обнародов...
3         Премии "Книга года" в 13 номинациях вручены на...
4         Сборная Португалии сыграла вничью с командой И...
                                ...                        
206632    Бывшей вице-президенту ювелирной компании Tiff...
206633    Минфин предлагает увеличить размер штрафа для ...
206634    Индекс потребительской уверенности россиян в п...
206635    Капитан женской сборной России по теннису Шами...
206636    Союз европейских футбольных ассоциаций (УЕФА) ...
Name: text, Length: 206636, dtype: object

In [51]:
t = []
text = 'У российских кинематографистов  появится новый'
for word in text.split():
    if len(word)<=3:
        continue
    p = morph.parse(word)[0]
    t.append(p.normal_form)

In [53]:
" ".join(t)

'российский кинематографист появиться новый'

In [ ]:
%%time
df['text_clean_normal'] = df.apply(lemmatize,axis=1)

In [54]:
import pickle

In [57]:
df[['text','text_clean_normal']].tail(10)

,text,text_clean_normal
206627,Государство выдвинет для избрания в совет дире...,государство выдвинуть избрание совет директор ...
206628,"Режиссер Василий Сигарев рассказал о том, что ...",режиссёр василий сигарёв рассказать добиться у...
206629,Руководителей компании Valve Software удивила ...,руководитель компания удивить информация второ...
206630,Бывший тренер сборной России Владимир Плющев п...,бывший тренер сборная россия владимир плющев п...
206631,Москвич Александр Морозевич одержал черными по...,москвич александр морозевич одержать чёрный по...
206632,Бывшей вице-президенту ювелирной компании Tiff...,бывший вица президент ювелирный компания ингри...
206633,Минфин предлагает увеличить размер штрафа для ...,минфин предлагать увеличить размер штраф должн...
206634,Индекс потребительской уверенности россиян в п...,индекс потребительский уверенность россиянин п...
206635,Капитан женской сборной России по теннису Шами...,капитан женский сборная россия теннис шамиль т...
206636,Союз европейских футбольных ассоциаций (УЕФА) ...,союз европейский футбольный ассоциация уефа пр...


### качество модели после лемматизации

In [58]:
text_transformer = TfidfVectorizer(stop_words=stop_russian, ngram_range=(1,1), lowercase=True, max_features=10000)
text_norm = text_transformer.fit_transform(df['text_clean_normal'])
X_train, X_test, y_train, y_test = train_test_split(text_norm, df['topic_label'], test_size=0.20, random_state=rs)

In [59]:
clf_norm = tree.DecisionTreeClassifier()
clf_norm.fit(X_train,y_train)
pred_norm = clf_norm.predict(X_test)

In [60]:

print('Качество модели по метрике F1 после лемматизации', f1_score(y_test,pred_norm,average='weighted'))

Качество модели по метрике F1 после лемматизации 0.9013850129316823
